In [18]:
# import packages
from datetime import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pandas as pd

## export a SD file with ... ##
# time, lon, lat, QL, QS, qs, qa, SST, T, V
sd0 = pd.read_csv('data/sd/avg-at30-2017.csv')

p = sd0['p'] # hPa
T = sd0['T']+273.15 # C to K
SST = sd0['SST']+273.15 # C to K
RH = sd0['RH'] # %

# conversions for sea
# sea saturation vapor pressure : Wallace and Hobbs, Second Edition (pg. 99)
es = 6.11*np.exp((2.50*10**6*18.016/(1000*8.3145))*((1/273)-(1/SST))) #hPa
e = es * (RH/100.0) # hPa
qs = 0.622*(e/(p-e))*1000 #g/kg

# conversions for air
es = 6.11*np.exp((2.50*10**6*18.016/(1000*8.3145))*((1/273)-(1/T))) # air saturation vapor pressure : Wallace and Hobbs, Second Edition (pg. 99)
e = es * (RH/100.0) # vapor pressure : Wallace and Hobbs, Second Edition (pg. 82)
qa = 0.622*(e/(p-e))*1000 # specific humidity of atmosphere : Wallace and Hobbs (pg. 80)

t = sd0['datetime']
lon = sd0['longitude']
lat = sd0['latitude']
QL = sd0['QL']
QS = sd0['QS']
V = sd0['V']

sd = np.array([t,lon,lat, QL,qs,qa,QS,SST,T,V,p])
sd = pd.DataFrame(sd.T,columns=['datetime','lon','lat','QL','qa','qs','QS','SST','T','V','p'])
sd.to_csv('data/sd/avg-at30-2017.csv', encoding='utf-8', index=False)

In [18]:
# import packages
from datetime import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pandas as pd

sd = pd.read_csv('data/e5/sd-for-e5-2017.csv')

t = sd['datetime']
lon = sd['lon']
lat = sd['lat']
QL = sd['QL']
QS = sd['QS']
qs = sd['qs']
qa = sd['qa']
SST = sd['SST']
T = sd['T']
V = sd['V']

land = pd.read_csv('check-land/e5land.csv')
land_lat = land['lat']
land_lon = land['lng']
land = [land_lon,land_lat]
land = np.array(land)

In [19]:
import math
from typing import Tuple

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance between two points on the Earth's surface
    using the Haversine formula.
    
    lon1, lat1: Longitude and latitude of the first point (in degrees).
    lon2, lat2: Longitude and latitude of the second point (in degrees).
    
    Returns the distance in kilometers.
    """
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = 6371 * c  # Earth's radius in kilometers
    return distance
def find_nearest(coordinates, target_point):
    closest_index = 0
    closest_distance = haversine(target_point[0], target_point[1], coordinates[0][0], coordinates[0][1])
    for i, coordinate in enumerate(coordinates):
        distance = haversine(target_point[0], target_point[1], coordinate[0], coordinate[1])
        if distance < closest_distance:
            closest_index = i
            closest_distance = distance
    return closest_index
def hours_to_unix_timestamp(hours_array):
    # Set the starting date as May 1, 2019
    start_date = np.datetime64('1899-12-31T23:30:00')
    # Calculate the time difference in seconds for the entire array
    time_difference_seconds = (hours_array * 3600)
    # Convert the time differences to timedelta64 objects
    time_difference_timedelta = time_difference_seconds.astype('timedelta64[s]')
    # Add the time difference to the starting date
    result_dates = start_date + time_difference_timedelta
    # Convert the result dates to Unix timestamps
    unix_timestamps = (result_dates - np.datetime64('1970-01-01')) / np.timedelta64(1, 's')
    return unix_timestamps
# pretty ones from Eli
def solution(X1: np.ndarray, X2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Params:
        X1: (1D array_like)
        X2: (1D array_like)
    Returns:
        X1_indices where value exists in X2 as well
        X2_indices where value exists in X1 as well
    Note: the returned indices array are ordered smallest to greatest. by the value they correspond to
    that is to say X1[X1_indices] is a sorted list, u could do X1[X1_indices.sort()] to get the values in 
    the order they appear in the orignal X1
    """
    inter = np.intersect1d(X1, X2)
    def helper(inter: np.ndarray, x: np.ndarray):
        sorter = np.argsort(x)
        searchsorted_left = np.searchsorted(x, inter, sorter=sorter,side='left')
        searchsorted_right = np.searchsorted(x, inter, sorter=sorter,side='right')
        values = vrange(searchsorted_left, searchsorted_right) 
        return sorter[values] # optional to sort this if u care?
    return helper(inter, X1), helper(inter, X2)
def vrange(starts: np.ndarray, stops: np.ndarray):
    """Create concatenated ranges of integers for multiple start/stop
    Parameters:
        starts (1-D array_like): starts for each range
        stops (1-D array_like): stops for each range (same shape as starts)
    Returns:
        numpy.ndarray: concatenated ranges
    For example:
        >>> starts = [1, 3, 4, 6]
        >>> stops  = [1, 5, 7, 6]
        >>> vrange(starts, stops)
        array([3, 4, 4, 5, 6])
    """
    stops = np.asarray(stops)
    l = stops - starts # Lengths of each range.
    return np.repeat(stops - l.cumsum(), l) + np.arange(l.sum())

prd = 'raw-data/e5/2017.nc'
f = nc.Dataset(prd,mode='r')

tprd = f.variables['time'][:].astype(np.int64)
lonprd = f.variables['longitude'][:] # degrees
latprd = f.variables['latitude'][:] # degrees
QLprd = f.variables['slhf'][:]/-3600 # W*m^-2
TDprd = f.variables['d2m'][:] # K
QSprd = f.variables['sshf'][:]/-3600 # W*m^-2
SSTprd = f.variables['sst'][:] # K
Tprd = f.variables['t2m'][:] # K
uprd = f.variables['u10'][:] # m*s^-1
vprd = f.variables['v10'][:] # m*s^-1
pprd = f.variables['sp'][:] # Pa

tprd = hours_to_unix_timestamp(tprd)
Vprd = np.sqrt(uprd**2+vprd**2)

In [20]:
# organize coordinates
location = np.stack((lon,lat),axis=1) # set saildrone coordinates

gridlocation = [] # set MERRA2 coordinates
grididx = []
for idx_lon, lon in enumerate(lonprd):
    for idx_lat,lat in enumerate(latprd):
        gridlocation.append([lon,lat])
        grididx.append([idx_lon,idx_lat])
gridlocation = np.asarray(gridlocation)
grididx = np.asarray(grididx)

lonidx = []
latidx = []
tttidx = []
for idx,loc in enumerate(location):
    k = find_nearest(gridlocation, loc)
    # Initialize a flag to check if any match is found
    match_found = False
    for land_coord in land.T:
        if gridlocation[k][0] == land_coord[0] and gridlocation[k][1] == land_coord[1]:
            # Set the flag to True if a match is found
            match_found = True
            break
    # Append indices only if no match is found
    if not match_found:
        lonidx.append(grididx[k, 0])
        latidx.append(grididx[k, 1])
        tttidx.append(idx)

# organize times
t = np.copy(t[tttidx])
[idxprd,idxsd] = solution(tprd,t)
idxprd = np.array(idxprd)
tidxprd = np.copy(tprd[idxprd])
QLidxprd = np.copy(QLprd[idxprd])
QSidxprd = np.copy(QSprd[idxprd])
Tidxprd = np.copy(Tprd[idxprd])
SSTidxprd = np.copy(SSTprd[idxprd])
TDidxprd = np.copy(TDprd[idxprd])
Vidxprd = np.copy(Vprd[idxprd])
pidxprd = np.copy(pprd[idxprd])

In [21]:
# interpolate MERRA2 variables
QL0 = []
TD0 = []
QS0 = []
SST0 = []
T0 = []
SST0 = []
V0 = []
p0 = []

for tt in t:
    for idx,ttprd in enumerate(tidxprd):
        if tt == ttprd:
            QL0.append(QLidxprd[idx])
            TD0.append(TDidxprd[idx])
            QS0.append(QSidxprd[idx])
            SST0.append(SSTidxprd[idx])
            T0.append(Tidxprd[idx])
            V0.append(Vidxprd[idx])
            p0.append(pidxprd[idx])
QL0 = np.array(QL0)
TD0 = np.array(TD0)
QS0 = np.array(QS0)
SST0 = np.array(SST0)
T0 = np.array(T0)
V0 = np.array(V0)
p0 = np.array(p0)

tidx = np.arange(0,len(t),1)
QL = []
TD = []
QS = []
SST = []
T = []
SST = []
V = []
p = []

for j in tidx:
    # first closest
    QL.append(QL0[j,latidx[j],lonidx[j]])
    TD.append(TD0[j,latidx[j],lonidx[j]])
    QS.append(QS0[j,latidx[j],lonidx[j]])
    SST.append(SST0[j,latidx[j],lonidx[j]])
    T.append(T0[j,latidx[j],lonidx[j]])
    V.append(V0[j,latidx[j],lonidx[j]])
    p.append(p0[j,latidx[j],lonidx[j]])

latprd = np.array(latprd[latidx])
lonprd = np.array(lonprd[lonidx])
p = np.array(p)
TD = np.array(TD)-273.15
SST = np.array(SST)
T = np.array(T)

e = 6.11*10**((7.5*TD)/(237.7+TD))
qa = 1000*0.622*e/(p/100-e)

es = 6.11*2.71828**(5420*(1/273-1/(T))) # air saturation vapor pressure OK
rh = 100*e/es

es = 6.11*2.71828**(5420*(1/273-1/(SST))) # sea saturation vapor pressure OK
e = es*rh/100
qs = 1000*0.622*e/(p/100-e)

lon = np.copy(sd['lon'][tttidx])
lat = np.copy(sd['lat'][tttidx])
prd = np.array([t,lon,lat,lonprd,latprd,QL,qs,qa,QS,SST,T,V])
prd = prd.T
prd = np.ma.masked_where(prd == -32767, prd)
prd = np.ma.compress_rows(prd)
prd = pd.DataFrame(prd,columns=['datetime','lon','lat','lon-prd','lat-prd','QL','qs','qa','QS','SST','T','V'])
prd.to_csv('data/e5/e5-2017.csv', encoding='utf-8', index=False)